In [1]:
import pandas as pd
import time
import numpy as np
np.seterr(invalid='ignore')
import nbimporter
from PPP import PPP

prevGames = ['Game1','Game2']
Opp = 'Nuggets'
Team = 'Heat'
series = 'Finals'

In [2]:
players = ['adebayo', 'butler', 'highsmith', 'love', 'lowry', 'martin', 'robinson', 'strus', 'vincent', 'zeller', 'team', 'opp']

In [31]:
def load_data(players):
    
    # Establish dictionaries for new game data and overall series/season data
    new_player_data = {}
    ovr_player_data = {}
    ovr_series_player_data = {}
    team_opp = ['team', 'opp']
    
    for game in prevGames: # Loop over all games in series/season
        
        for player in players: # Loop over all players on roster
            
            if player == 'team': # Add all player data together to create team file for game

                try: # to load a team file for the game (helpful if already added team file, and/or rerunning for Opp file)
                    newplayerFile = (f'Team/{game + Opp}Team.csv')
                    newplayer = pd.read_csv(newplayerFile)
                    newplayer = newplayer.dropna()
                    newplayer = newplayer.set_index('Shot Type')
                    newplayer = newplayer.astype('int')
                    new_player_data[player] = newplayer # Add the existing new team file to the new_player_data dictionary
                except(FileNotFoundError):
                    # Load an empty df
                    newplayerFile = ('Team/empty.csv')
                    newplayer = pd.read_csv(newplayerFile)
                    newplayer = newplayer.set_index('Shot Type')

                    for key, df in new_player_data.items(): # Loop over the new_player_data and add together all player dataframes
                        if key not in team_opp: # Exclude the team and opponent dataframes
                            newplayer = newplayer.add(df)

                    new_player_data['team'] = newplayer # Add the created new team file to new_player_data dictionary
                    newplayer.to_csv(f'../{Opp}/Opp/{game + Team}Opp.csv') # Save team file to opponent folder for their opponent stats
                    newplayer.to_csv(f'Team/{game + Opp}Team.csv') # Save team file to team folder
                    
                # Load overall data for the team
                try:
                    ovrplayerFile = (f'!Data/Team.csv')
                    ovrplayer = pd.read_csv(ovrplayerFile)
                    ovrplayer = ovrplayer.dropna()
                    ovrplayer = ovrplayer.set_index('Shot Type')
                    ovrplayer = ovrplayer.astype('int')
                    ovr_player_data[player] = ovrplayer # Add the overall team data to the ovr_player_data dictionary
                except(FileNotFoundError):
                    ovrplayer = newplayer
                    ovr_player_data[player] = ovrplayer
                    ovrplayer.to_csv(f'!Data/{player.title()}.csv')
                
                # Load overall series data for the player
                try: 
                    ovr_seriesplayerFile = (f'!Data/{series}/{player.title() + series}.csv')
                    ovr_seriesplayer = pd.read_csv(ovr_seriesplayerFile)
                    ovr_seriesplayer = ovr_seriesplayer.dropna()
                    ovr_seriesplayer = ovr_seriesplayer.set_index('Shot Type')
                    ovr_seriesplayer = ovr_seriesplayer.astype('int')
                    ovr_series_player_data[player] = ovr_seriesplayer # Add overall player data to the ovr_player_data dictionary
                # If there is no overall series data for the player, set the new player data as the overall series data and continue to the next player    
                except(FileNotFoundError):
                    ovr_seriesplayer = newplayer
                    ovr_series_player_data[player] = ovr_seriesplayer
                    ovr_seriesplayer.to_csv(f'!Data/{series}/{player.title() + series}.csv')
                    continue
                    
                if newplayer.columns.equals(ovrplayer.columns) and newplayer.columns.equals(ovr_seriesplayer.columns): # Make sure the new and overall dataframes can be added without error
                    ovrplayer = ovrplayer.add(newplayer)
                    ovr_player_data[player] = ovrplayer
                    ovrplayer.to_csv(f'!Data/{player.title()}.csv') # Write the updated overall player file to the overall data directory
                    if game == 'Game1':
                        continue

                    ovr_seriesplayer = ovr_seriesplayer.add(newplayer)
                    ovr_series_player_data[player] = ovr_seriesplayer
                    ovr_seriesplayer.to_csv(f'!Data/{series}/{player.title() + series}.csv' )# Write the updated overall series player file to the overall series data directory
                else:
                    print(f"{player.title()}'s columns do not match")  
                continue

            if player == 'opp': # Load the opponent file and add the new game data to the overall opponent file if necessary

                try: # to load the new opponent file (if running this teams data first, there will be no opponent file!!)
                    newplayerFile = (f'Opp/{game + Opp}Opp.csv')
                    newplayer = pd.read_csv(newplayerFile)
                    newplayer = newplayer.dropna()
                    newplayer = newplayer.set_index('Shot Type')
                    newplayer = newplayer.astype('int')
                    new_player_data[player] = newplayer # Add the new opponent file to the new_player_data dictionary
                except(FileNotFoundError):
                    print(f'No {player.title()} file found.')
                    continue
                    
                # Load overall data for the opponent
                try:
                    ovrplayerFile = (f'!Data/Opp.csv')
                    ovrplayer = pd.read_csv(ovrplayerFile)
                    ovrplayer = ovrplayer.dropna()
                    ovrplayer = ovrplayer.set_index('Shot Type')
                    ovrplayer = ovrplayer.astype('int')
                    ovr_player_data[player] = ovrplayer # Add the overall opponent file to the ovr_player_data dictionary
                except(FileNotFoundError):
                    if newplayer:
                        ovrplayer = newplayer
                        ovr_player_data[player] = ovrplayer
                        ovrplayer.to_csv(f'!Data/{player.title()}.csv')
                    else:
                        print(f'No overall Opponent file found.')
                        continue
                        
                # Load overall series data for the player
                try: 
                    ovr_seriesplayerFile = (f'!Data/{series}/{player.title() + series}.csv')
                    ovr_seriesplayer = pd.read_csv(ovr_seriesplayerFile)
                    ovr_seriesplayer = ovr_seriesplayer.dropna()
                    ovr_seriesplayer = ovr_seriesplayer.set_index('Shot Type')
                    ovr_seriesplayer = ovr_seriesplayer.astype('int')
                    ovr_series_player_data[player] = ovr_seriesplayer # Add overall player data to the ovr_player_data dictionary
                # If there is no overall series data for the player, set the new player data as the overall series data and continue to the next player    
                except(FileNotFoundError):
                    ovr_seriesplayer = newplayer
                    ovr_series_player_data[player] = ovr_seriesplayer
                    ovr_seriesplayer.to_csv(f'!Data/{series}/{player.title() + series}.csv')
                    continue

                if newplayer.columns.equals(ovrplayer.columns) and newplayer.columns.equals(ovr_seriesplayer.columns): # Make sure the new and overall dataframes can be added without error
                    ovrplayer = ovrplayer.add(newplayer)
                    ovr_player_data[player] = ovrplayer
                    ovrplayer.to_csv(f'!Data/{player.title()}.csv') # Write the updated overall player file to the overall data directory
                    if game == 'Game1':
                        continue

                    ovr_seriesplayer = ovr_seriesplayer.add(newplayer)
                    ovr_series_player_data[player] = ovr_seriesplayer
                    ovr_seriesplayer.to_csv(f'!Data/{series}/{player.title() + series}.csv' )# Write the updated overall series player file to the overall series data directory
                else:
                    print(f"{player.title()}'s columns do not match")  
                continue
                
            ### Loading/Adding the player files (first part of the loop)###
            try:
                newplayerFile = (f'{player.title()}/{game + Opp + player.title()}.csv') # Search the team directory for the player, then concat the game/opp/player to find csv for game
                newplayer = pd.read_csv(newplayerFile)
                newplayer = newplayer.dropna()
                newplayer = newplayer.set_index('Shot Type')
                newplayer = newplayer.astype('int')
                new_player_data[player] = newplayer # Add the new player data to the new_player_data dictionary
            except(FileNotFoundError):
                print(f'No {player.title()} file found for {game}.')
                try: # to load an overall file for the player
                    ovrplayerFile = (f'!Data/{player.title()}.csv')
                    ovrplayer = pd.read_csv(ovrplayerFile)
                    ovrplayer = ovrplayer.dropna()
                    ovrplayer = ovrplayer.set_index('Shot Type')
                    ovrplayer = ovrplayer.astype('int')
                    ovr_player_data[player] = ovrplayer # Add overall player data to the ovr_player_data dictionary
                    continue
                except(FileNotFoundError):
                    print(f'No data for {player.title()}.')
                    continue # If no player data is found, skip and go to the next player (edit for later)
                    
                try: # to load an overall player file for the series
                    ovr_seriesplayerFile = (f'!Data/{player.title()}.csv')
                    ovr_seriesplayer = pd.read_csv(ovr_seriesplayerFile)
                    ovr_seriesplayer = ovr_seriesplayer.dropna()
                    ovr_seriesplayer = ovr_seriesplayer.set_index('Shot Type')
                    ovr_seriesplayer = ovr_seriesplayer.astype('int')
                    ovr_series_player_data[player] = ovr_seriesplayer # Add overall player data to the ovr_series_player_data dictionary
                    continue
                except(FileNotFoundError):
                    print(f'No {series} series data for {player.title()}.')
                    continue # If no player data is found, skip and go to the next player (edit for later)
                    
            # Load overall data for the player
            try: 
                ovrplayerFile = (f'!Data/{player.title()}.csv')
                ovrplayer = pd.read_csv(ovrplayerFile)
                ovrplayer = ovrplayer.dropna()
                ovrplayer = ovrplayer.set_index('Shot Type')
                ovrplayer = ovrplayer.astype('int')
                ovr_player_data[player] = ovrplayer # Add overall player data to the ovr_player_data dictionary
            # If there is no overall data for the player, set the new player data as the overall and continue to the next player    
            except(FileNotFoundError):
                ovrplayer = newplayer
                ovr_player_data[player] = ovrplayer
                ovrplayer.to_csv(f'!Data/{player.title()}.csv')
              
            # Load overall series data for the player
            try: 
                ovr_seriesplayerFile = (f'!Data/{series}/{player.title() + series}.csv')
                ovr_seriesplayer = pd.read_csv(ovr_seriesplayerFile)
                ovr_seriesplayer = ovr_seriesplayer.dropna()
                ovr_seriesplayer = ovr_seriesplayer.set_index('Shot Type')
                ovr_seriesplayer = ovr_seriesplayer.astype('int')
                ovr_series_player_data[player] = ovr_seriesplayer # Add overall player data to the ovr_player_data dictionary
            # If there is no overall series data for the player, set the new player data as the overall series data and continue to the next player    
            except(FileNotFoundError):
                ovr_seriesplayer = newplayer
                ovr_series_player_data[player] = ovr_seriesplayer
                ovr_seriesplayer.to_csv(f'!Data/{series}/{player.title() + series}.csv')
                continue
                
            if newplayer.columns.equals(ovrplayer.columns) and newplayer.columns.equals(ovr_seriesplayer.columns): # Make sure the new and overall dataframes can be added without error
                ovrplayer = ovrplayer.add(newplayer)
                ovr_player_data[player] = ovrplayer
                ovrplayer.to_csv(f'!Data/{player.title()}.csv') # Write the updated overall player file to the overall data directory
                if game == 'Game1':
                    continue
                    
                ovr_seriesplayer = ovr_seriesplayer.add(newplayer)
                ovr_series_player_data[player] = ovr_seriesplayer
                ovr_seriesplayer.to_csv(f'!Data/{series}/{player.title() + series}.csv' )# Write the updated overall series player file to the overall series data directory
            else:
                print(f"{player.title()}'s columns do not match")  
            continue
            
    return new_player_data, ovr_player_data, ovr_series_player_data # Return both dictionaries as a tuple

new_player_data, ovr_player_data, ovr_series_player_data  = load_data(players) # Unpack the tuple of dictionaries into their respective names

No Love file found for Game1.


In [4]:
newButlerStats = PPP(new_player_data['butler'])
newButlerStats.to_csv(f'Butler/PPP Stats/{prevGames[-1] + Opp}.csv')
newButlerStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.3,42.1,1,50.0,0.29,31.8,16.7,5,20.0,1,0.0,2.33,62.5,83.3,2,50.0,4,100.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,3.0,0,0.0,0.0,4.5,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.58,32.6,0,66.7,1.48,30.7,60.0,4,75.0,1,0.0,1.75,36.4,75.0,2,100.0,2,50.0
TRANSITION,0.0,3.0,0,0.0,0.0,4.5,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,1.23,7.4,0,50.0,1.23,11.1,50.0,1,100.0,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,3.0,6.1,0,100.0,3.0,9.1,100.0,0,N/A,2,100.0,N/A,0.0,N/A,0,N/A,0,N/A


In [5]:
newButlerSeriesStats = PPP(ovr_series_player_data['butler'])
newButlerSeriesStats.to_csv(f'Butler/PPP Stats/{prevGames[-1] + Opp}.csv')
newButlerSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.87,38.8,1,33.3,0.29,37.8,15.4,11,18.2,2,0.0,1.8,40.4,62.5,3,33.3,5,80.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,1.7,0,0.0,0.0,2.7,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,0.0,1.7,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,4.5,0.0,0,N/A,1,0.0
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.58,30.1,0,62.5,1.68,21.0,66.7,4,75.0,2,50.0,1.5,45.5,60.0,3,100.0,7,42.9
TRANSITION,0.67,5.1,0,33.3,0.67,8.1,33.3,3,33.3,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.87,5.8,1,50.0,0.87,9.3,50.0,1,100.0,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,3.0,3.4,0,100.0,3.0,5.4,100.0,0,N/A,2,100.0,N/A,0.0,N/A,0,N/A,0,N/A


In [6]:
butlerStats = PPP(ovr_player_data['butler'])
butlerStats.to_csv('!PPP Stats/butlerStats.csv')
butlerStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.87,29.9,7,37.1,0.75,28.3,31.0,50,34.0,8,12.5,1.1,33.0,48.4,14,42.9,17,52.9
PNR Screener,0.0,0.6,0,0.0,0.0,0.4,0.0,1,0.0,0,N/A,0.0,0.9,0.0,0,N/A,1,0.0
DHO Ball Handler,1.0,0.9,0,33.3,1.0,1.3,33.3,2,0.0,1,100.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,0.71,2.0,1,33.3,N/A,0.0,N/A,0,N/A,0,N/A,0.71,6.2,33.3,2,50.0,4,25.0
DBL Ball Handler,0.0,1.2,0,0.0,0.0,0.4,0.0,0,N/A,1,0.0,0.0,2.7,0.0,0,N/A,3,0.0
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.15,43.4,7,47.3,0.94,42.3,38.3,65,38.5,16,37.5,1.56,45.5,62.5,14,78.6,34,55.9
TRANSITION,1.45,6.4,2,76.5,1.36,7.0,69.2,13,69.2,0,N/A,1.7,5.2,100.0,2,100.0,2,100.0
Attacking Closeouts,0.67,3.0,4,50.0,0.67,4.5,50.0,5,60.0,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,2.83,1.7,0,100.0,2.83,2.6,100.0,1,100.0,5,100.0,N/A,0.0,N/A,0,N/A,0,N/A


In [7]:
newAdebayoStats = PPP(new_player_data['adebayo'])
newAdebayoStats.to_csv(f'Adebayo/PPP Stats/{prevGames[-1] + Opp}.csv')
newAdebayoStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.0,4.3,0,0.0,0.0,6.2,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,1.48,29.4,1,75.0,1.62,27.0,66.7,3,66.7,0,N/A,1.23,34.9,100.0,1,100.0,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,1.23,21.2,1,66.7,2.0,12.5,100.0,2,100.0,0,N/A,0.69,41.1,0.0,0,N/A,1,0.0
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.57,30.4,0,28.6,0.67,37.5,33.3,6,33.3,0,N/A,0.0,14.3,0.0,0,N/A,1,0.0
TRANSITION,2.27,3.8,0,N/A,2.27,5.5,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [8]:
newAdebayoSeriesStats = PPP(ovr_series_player_data['adebayo'])
newAdebayoSeriesStats.to_csv(f'Adebayo/PPP Stats/{prevGames[-1] + Opp}.csv')
newAdebayoSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.0,1.9,0,0.0,0.0,2.5,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,1.19,22.2,3,71.4,1.18,23.3,66.7,6,66.7,0,N/A,1.23,18.8,100.0,1,100.0,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,0.92,20.5,1,44.4,0.67,15.0,33.3,6,33.3,0,N/A,1.23,37.5,66.7,2,100.0,1,0.0
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.95,35.8,1,50.0,1.0,40.0,50.0,16,50.0,0,N/A,0.67,23.1,50.0,1,100.0,1,0.0
TRANSITION,1.39,5.4,0,50.0,2.27,2.2,N/A,0,N/A,0,N/A,1.0,15.4,50.0,1,100.0,1,0.0
Attacking Closeouts,2.0,1.9,0,100.0,2.0,2.5,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,2.0,1.9,0,100.0,2.0,2.5,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [9]:
adebayoStats = PPP(ovr_player_data['adebayo'])
adebayoStats.to_csv('!PPP Stats/adebayoStats.csv')
adebayoStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.24,2.1,0,33.3,1.24,3.0,33.3,3,33.3,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,1.08,18.0,16,74.2,1.03,20.9,70.4,27,70.4,0,N/A,1.27,11.4,100.0,1,100.0,3,100.0
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,0.89,18.9,2,37.0,0.43,9.9,18.8,16,18.8,0,N/A,1.16,39.5,46.7,12,66.7,18,33.3
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.03,41.9,10,53.0,0.96,47.1,48.1,79,48.1,0,N/A,1.28,30.1,71.4,16,81.2,5,40.0
TRANSITION,1.42,7.2,1,64.7,1.45,3.6,80.0,5,80.0,0,N/A,1.4,15.5,58.3,5,100.0,7,28.6
Attacking Closeouts,1.5,1.5,0,75.0,1.5,2.1,75.0,4,75.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,2.0,1.1,0,100.0,2.0,1.6,100.0,3,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [10]:
newLoveStats = PPP(new_player_data['love'])
newLoveStats.to_csv(f'Love/PPP Stats/{prevGames[-1] + Opp}.csv')
newLoveStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.0,14.3,0,0.0,0.0,14.3,0.0,0,N/A,1,0.0,N/A,N/A,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Screener,3.0,14.3,0,100.0,3.0,14.3,100.0,0,N/A,1,100.0,N/A,N/A,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
TRANSITION,2.27,12.6,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,2.27,N/A,N/A,0,N/A,0,N/A
Attacking Closeouts,1.5,28.6,0,50.0,1.5,28.6,50.0,0,N/A,2,50.0,N/A,N/A,N/A,0,N/A,0,N/A
Catch & Shoot,0.0,28.6,0,0.0,0.0,28.6,0.0,0,N/A,2,0.0,N/A,N/A,N/A,0,N/A,0,N/A


In [11]:
LoveSeriesStats = PPP(ovr_series_player_data['love'])
LoveSeriesStats.to_csv(f'Love/PPP Stats/{prevGames[-1] + Opp}.csv')
LoveSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.0,14.3,0,0.0,0.0,14.3,0.0,0,N/A,1,0.0,N/A,N/A,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Screener,3.0,14.3,0,100.0,3.0,14.3,100.0,0,N/A,1,100.0,N/A,N/A,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
TRANSITION,2.27,12.6,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,2.27,N/A,N/A,0,N/A,0,N/A
Attacking Closeouts,1.5,28.6,0,50.0,1.5,28.6,50.0,0,N/A,2,50.0,N/A,N/A,N/A,0,N/A,0,N/A
Catch & Shoot,0.0,28.6,0,0.0,0.0,28.6,0.0,0,N/A,2,0.0,N/A,N/A,N/A,0,N/A,0,N/A


In [12]:
loveStats = PPP(ovr_player_data['love'])
loveStats.to_csv('!PPP Stats/loveStats.csv')
loveStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,0.0,6.7,1,0.0,0.0,4.3,0.0,1,0.0,0,N/A,0.0,14.3,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,3.3,1,N/A,N/A,0.0,N/A,0,N/A,0,N/A,0.0,14.3,N/A,0,N/A,0,N/A
DHO Screener,0.0,3.3,0,0.0,0.0,4.3,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.5,26.7,0,25.0,0.29,30.4,14.3,7,14.3,0,N/A,2.0,14.3,100.0,1,100.0,0,N/A
TRANSITION,1.6,16.7,0,60.0,1.5,17.4,50.0,0,N/A,4,50.0,2.0,14.3,100.0,1,100.0,0,N/A
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,2.0,23.3,1,83.3,1.83,26.1,80.0,1,100.0,4,75.0,3.0,14.3,100.0,0,N/A,1,100.0


In [13]:
newLowryStats = PPP(new_player_data['lowry'])
newLowryStats.to_csv(f'Lowry/PPP Stats/{prevGames[-1] + Opp}.csv')
newLowryStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.91,52.4,0,75.0,1.81,55.3,50.0,1,0.0,1,100.0,2.05,48.8,100.0,2,100.0,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,18.2,0,0.0,0.0,16.7,0.0,0,N/A,1,0.0,0.0,20.0,0.0,1,0.0,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.0,18.2,2,N/A,0.0,16.7,N/A,0,N/A,0,N/A,0.0,20.0,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,3.0,9.1,0,100.0,3.0,16.7,100.0,0,N/A,1,100.0,N/A,0.0,N/A,0,N/A,0,N/A


In [14]:
newLowrySeriesStats = PPP(ovr_series_player_data['lowry'])
newLowrySeriesStats.to_csv(f'Lowry/PPP Stats/{prevGames[-1] + Opp}.csv')
newLowrySeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,2.15,48.5,0,77.8,2.05,56.3,66.7,2,0.0,4,100.0,2.31,39.3,100.0,2,100.0,1,100.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.5,16.7,0,25.0,0.67,23.1,33.3,1,100.0,2,0.0,0.0,9.1,0.0,1,0.0,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.0,12.5,2,0.0,0.0,7.7,N/A,0,N/A,0,N/A,0.0,18.2,0.0,0,N/A,1,0.0
TRANSITION,2.0,8.3,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,2.0,18.2,100.0,2,100.0,0,N/A
Attacking Closeouts,0.0,4.2,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,9.1,0.0,0,N/A,1,0.0
Catch & Shoot,1.5,8.3,0,50.0,1.5,15.4,50.0,0,N/A,2,50.0,N/A,0.0,N/A,0,N/A,0,N/A


In [15]:
lowryStats = PPP(ovr_player_data['lowry'])
lowryStats.to_csv('!PPP Stats/lowryStats.csv')
lowryStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.28,47.2,8,54.0,1.17,53.1,45.7,18,22.2,17,70.6,1.48,39.6,73.3,10,70.0,5,80.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.73,8.2,0,36.4,0.89,11.8,44.4,4,100.0,5,0.0,0.0,3.4,0.0,1,0.0,1,0.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.91,11.5,5,60.0,0.75,10.5,75.0,4,75.0,0,N/A,1.08,12.8,50.0,2,100.0,4,25.0
TRANSITION,1.78,16.3,1,80.0,1.29,5.1,33.3,2,0.0,1,100.0,1.89,31.0,88.2,12,91.7,5,80.0
Attacking Closeouts,0.88,6.0,1,42.9,1.33,3.9,66.7,3,66.7,0,N/A,0.6,8.6,25.0,0,N/A,4,25.0
Catch & Shoot,0.9,7.5,0,30.0,0.9,13.2,30.0,0,N/A,10,30.0,N/A,0.0,N/A,0,N/A,0,N/A


In [16]:
newMartinStats = PPP(new_player_data['martin'])
newMartinStats.to_csv(f'Martin/PPP Stats/{prevGames[-1] + Opp}.csv')
newMartinStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,0.0,25.0,0,0.0,0.0,25.0,0.0,1,0.0,0,N/A,0.0,25.0,0.0,0,N/A,1,0.0
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.0,25.0,1,0.0,0.0,50.0,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,1.5,25.0,0,50.0,N/A,0.0,N/A,0,N/A,0,N/A,1.5,50.0,50.0,0,N/A,2,50.0
Catch & Shoot,3.0,12.5,0,100.0,3.0,25.0,100.0,0,N/A,1,100.0,N/A,0.0,N/A,0,N/A,0,N/A


In [17]:
newMartinSeriesStats = PPP(ovr_series_player_data['martin'])
newMartinSeriesStats.to_csv(f'Martin/PPP Stats/{prevGames[-1] + Opp}.csv')
newMartinSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.0,11.8,0,0.0,0.0,18.2,0.0,2,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,5.9,0,0.0,0.0,9.1,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,0.0,17.6,0,0.0,0.0,9.1,0.0,1,0.0,0,N/A,0.0,33.3,0.0,1,0.0,1,0.0
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.0,17.6,1,0.0,0.0,27.3,0.0,1,0.0,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.75,23.5,0,25.0,0.0,9.1,0.0,1,0.0,0,N/A,1.0,50.0,33.3,0,N/A,3,33.3
Catch & Shoot,2.0,17.6,0,66.7,2.0,27.3,66.7,0,N/A,3,66.7,N/A,0.0,N/A,0,N/A,0,N/A


In [18]:
martinStats = PPP(ovr_player_data['martin'])
martinStats.to_csv('!PPP Stats/martinStats.csv')
martinStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.36,7.7,1,20.0,0.36,9.6,20.0,10,20.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,0.67,2.1,0,33.3,0.67,2.6,33.3,1,100.0,2,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,1.0,4.2,0,33.3,1.0,5.2,33.3,4,0.0,2,100.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,0.75,5.6,0,25.0,0.0,0.9,0.0,1,0.0,0,N/A,0.86,25.9,28.6,3,0.0,4,50.0
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.62,18.2,3,31.8,0.7,19.9,33.3,13,53.8,8,0.0,0.0,11.1,0.0,0,N/A,1,0.0
TRANSITION,1.13,11.2,3,66.7,1.09,11.2,60.0,9,66.7,1,0.0,1.33,11.1,100.0,2,100.0,0,N/A
Attacking Closeouts,1.39,18.2,0,60.0,1.43,16.4,66.7,14,71.4,4,50.0,1.29,25.9,42.9,0,N/A,7,42.9
Catch & Shoot,1.94,24.6,0,65.7,1.91,29.6,64.7,1,100.0,33,63.6,3.0,3.7,100.0,0,N/A,1,100.0


In [19]:
newRobinsonStats = PPP(new_player_data['robinson'])
newRobinsonStats.to_csv(f'Robinson/PPP Stats/{prevGames[-1] + Opp}.csv')
newRobinsonStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
ISOLATION,3.0,16.7,0,100.0,3.0,16.7,100.0,0,N/A,1,100.0,N/A,N/A,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
Attacking Closeouts,2.05,40.7,0,100.0,2.05,40.7,100.0,1,100.0,1,100.0,N/A,N/A,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A


In [20]:
newRobinsonSeriesStats = PPP(ovr_series_player_data['robinson'])
newRobinsonSeriesStats.to_csv(f'Robinson/PPP Stats/{prevGames[-1] + Opp}.csv')
newRobinsonSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.0,5.9,1,N/A,0.0,7.7,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,11.8,0,0.0,0.0,7.7,0.0,0,N/A,1,0.0,0.0,25.0,0.0,0,N/A,1,0.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,3.0,5.9,0,100.0,3.0,7.7,100.0,0,N/A,1,100.0,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,0.0,5.9,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,25.0,0.0,1,0.0,0,N/A
Attacking Closeouts,1.65,32.0,0,80.0,1.45,26.5,66.7,1,100.0,2,50.0,2.0,50.0,100.0,2,100.0,0,N/A
Catch & Shoot,1.0,17.6,0,33.3,1.0,23.1,33.3,0,N/A,3,33.3,N/A,0.0,N/A,0,N/A,0,N/A


In [21]:
robinsonStats = PPP(ovr_player_data['robinson'])
robinsonStats.to_csv('!PPP Stats/robinsonStats.csv')
robinsonStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.29,13.7,3,63.6,0.8,13.3,42.9,2,50.0,5,40.0,2.5,14.8,100.0,2,100.0,2,100.0
PNR Screener,0.0,1.0,0,0.0,0.0,1.3,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.75,23.4,2,33.3,0.81,21.3,31.2,5,40.0,11,27.3,0.63,29.2,40.0,1,100.0,4,25.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.71,6.9,1,33.3,0.71,9.3,33.3,3,33.3,3,33.3,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,0.5,7.8,1,28.6,0.67,4.0,33.3,1,100.0,2,0.0,0.4,18.5,25.0,4,25.0,0,N/A
Attacking Closeouts,1.38,17.1,1,68.8,1.24,8.6,50.0,1,100.0,5,40.0,1.45,40.7,80.0,9,88.9,1,0.0
Catch & Shoot,1.4,14.7,1,50.0,1.4,20.0,50.0,0,N/A,14,50.0,N/A,0.0,N/A,0,N/A,0,N/A


In [22]:
newStrusStats = PPP(new_player_data['strus'])
newStrusStats.to_csv(f'Strus/PPP Stats/{prevGames[-1] + Opp}.csv')
newStrusStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,3.0,13.3,0,100.0,3.0,18.2,100.0,0,N/A,2,100.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,1.0,26.7,0,50.0,0.0,9.1,0.0,0,N/A,1,0.0,1.33,75.0,66.7,2,100.0,1,0.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,1.72,15.5,0,100.0,1.52,12.0,N/A,0,N/A,0,N/A,2.0,25.0,100.0,1,100.0,0,N/A
Catch & Shoot,0.5,40.0,0,16.7,0.5,54.5,16.7,0,N/A,6,16.7,N/A,0.0,N/A,0,N/A,0,N/A


In [23]:
newStrusSeriesStats = PPP(ovr_series_player_data['strus'])
newStrusSeriesStats.to_csv(f'Strus/PPP Stats/{prevGames[-1] + Opp}.csv')
newStrusSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.0,3.6,0,0.0,0.0,4.8,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,3.0,7.1,0,100.0,3.0,9.5,100.0,0,N/A,2,100.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.67,21.4,0,33.3,0.0,4.8,0.0,0,N/A,1,0.0,0.8,71.4,40.0,4,50.0,1,0.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,1.39,15.4,0,66.7,0.86,11.0,0.0,0,N/A,1,0.0,2.0,28.6,100.0,2,100.0,0,N/A
Catch & Shoot,0.23,46.4,0,7.7,0.23,61.9,7.7,0,N/A,13,7.7,N/A,0.0,N/A,0,N/A,0,N/A


In [24]:
strusStats = PPP(ovr_player_data['strus'])
strusStats.to_csv('!PPP Stats/strusStats.csv')
strusStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.0,16.8,2,44.4,1.0,16.7,42.9,6,33.3,8,50.0,1.0,17.4,50.0,3,66.7,1,0.0
PNR Screener,3.0,1.7,0,100.0,3.0,2.1,100.0,0,N/A,2,100.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.55,18.4,0,19.0,0.67,9.4,22.2,0,N/A,9,22.2,0.47,56.0,16.7,10,20.0,2,0.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.0,2.5,0,0.0,0.0,3.1,0.0,1,0.0,2,0.0,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,2.05,7.4,0,100.0,2.06,8.1,100.0,5,100.0,1,100.0,2.0,4.3,100.0,1,100.0,0,N/A
Attacking Closeouts,1.21,11.1,1,60.0,0.87,9.6,33.3,3,66.7,3,0.0,2.0,17.4,100.0,4,100.0,0,N/A
Catch & Shoot,0.55,33.5,0,17.9,0.54,40.6,17.9,0,N/A,39,17.9,1.14,3.8,N/A,0,N/A,0,N/A


In [25]:
newVincentStats = PPP(new_player_data['vincent'])
newVincentStats.to_csv(f'Vincent/PPP Stats/{prevGames[-1] + Opp}.csv')
newVincentStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.3,49.6,1,71.4,1.0,42.9,60.0,4,75.0,1,0.0,2.05,81.3,100.0,1,100.0,1,100.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,2.27,5.2,0,N/A,2.27,6.3,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,3.0,5.9,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,3.0,33.3,100.0,0,N/A,1,100.0
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.0,5.9,0,0.0,0.0,7.1,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,2.08,8.5,0,100.0,2.08,10.3,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,2.4,29.4,0,80.0,2.4,35.7,80.0,0,N/A,5,80.0,N/A,0.0,N/A,0,N/A,0,N/A


In [26]:
newVincentSeriesStats = PPP(ovr_series_player_data['vincent'])
newVincentSeriesStats.to_csv(f'Vincent/PPP Stats/{prevGames[-1] + Opp}.csv')
newVincentSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.4,41.1,1,66.7,1.1,35.7,55.6,6,66.7,3,33.3,1.86,53.7,83.3,3,100.0,3,66.7
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.52,9.7,0,0.0,1.06,6.7,0.0,0,N/A,1,0.0,0.0,16.7,0.0,2,0.0,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,1.67,7.5,0,66.7,N/A,0.0,N/A,0,N/A,0,N/A,1.67,25.0,66.7,1,100.0,2,50.0
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.0,2.5,0,0.0,0.0,3.6,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,2.08,3.6,0,100.0,2.08,5.1,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.67,7.5,0,33.3,1.0,7.1,50.0,2,50.0,0,N/A,0.0,8.3,0.0,0,N/A,1,0.0
Catch & Shoot,2.0,30.0,0,66.7,2.0,42.9,66.7,0,N/A,12,66.7,N/A,0.0,N/A,0,N/A,0,N/A


In [27]:
vincentStats = PPP(ovr_player_data['vincent'])
vincentStats.to_csv('!PPP Stats/vincentStats.csv')
vincentStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.33,40.1,4,60.9,1.24,39.0,55.6,28,64.3,17,41.2,1.54,42.9,73.7,10,90.0,9,55.6
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.54,9.6,1,14.3,0.72,7.9,14.3,1,0.0,6,16.7,0.29,14.0,14.3,7,14.3,0,N/A
DHO Screener,3.0,0.6,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,3.0,2.0,100.0,0,N/A,1,100.0
DBL Ball Handler,1.29,6.3,0,50.0,1.0,2.4,33.3,2,0.0,1,100.0,1.4,15.8,57.1,3,100.0,4,25.0
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.39,7.3,3,12.5,0.46,8.7,12.5,5,20.0,3,0.0,0.0,4.0,N/A,0,N/A,0,N/A
TRANSITION,1.78,4.2,0,66.7,2.46,2.0,100.0,1,100.0,1,100.0,1.43,9.8,50.0,1,100.0,3,33.3
Attacking Closeouts,0.76,9.1,0,28.6,0.85,9.5,30.0,9,33.3,1,0.0,0.5,8.0,25.0,1,100.0,3,0.0
Catch & Shoot,1.83,20.7,0,61.1,1.83,29.0,61.1,0,N/A,36,61.1,N/A,0.0,N/A,0,N/A,0,N/A


In [28]:
newZellerStats = PPP(new_player_data['zeller'])
newZellerStats.to_csv(f'Zeller/PPP Stats/{prevGames[-1] + Opp}.csv')
newZellerStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
PNR Screener,2.0,33.3,0,100.0,2.0,33.3,100.0,1,100.0,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
Attacking Closeouts,0.0,33.3,1,N/A,0.0,33.3,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A


In [29]:
newZellerSeriesStats = PPP(ovr_series_player_data['zeller'])
newZellerSeriesStats.to_csv(f'Zeller/PPP Stats/{prevGames[-1] + Opp}.csv')
newZellerSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,2.0,20.0,0,100.0,2.0,25.0,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.0,40.0,1,0.0,0.0,50.0,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [30]:
zellerStats = PPP(ovr_player_data['zeller'])
zellerStats.to_csv('!PPP Stats/zellerStats.csv')
zellerStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,1.0,24.0,1,60.0,1.33,17.6,66.7,3,66.7,0,N/A,0.67,37.5,50.0,1,100.0,1,0.0
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,2.0,4.0,0,100.0,2.0,5.9,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,0.0,3.5,0,N/A,0.0,5.2,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.0,20.0,1,0.0,0.0,29.4,0.0,4,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [33]:
newTeamStats = PPP(new_player_data['team'])
newTeamStats.to_csv(f'Team/PPP Stats/{prevGames[-1] + Opp}.csv')
newTeamStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.33,23.7,2,56.0,0.76,20.4,33.3,11,36.4,4,25.0,2.21,31.8,90.0,5,80.0,5,100.0
PNR Screener,1.84,7.7,1,85.7,2.05,8.1,83.3,4,75.0,2,100.0,1.23,6.6,100.0,1,100.0,0,N/A
DHO Ball Handler,0.76,6.2,0,28.6,0.52,4.3,0.0,1,0.0,2,0.0,1.0,10.8,50.0,3,66.7,1,0.0
DHO Screener,0.87,5.4,1,40.0,1.33,3.3,66.7,3,66.7,0,N/A,0.52,10.5,0.0,0,N/A,2,0.0
DBL Ball Handler,3.0,0.8,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,3.0,2.7,100.0,0,N/A,1,100.0
DBL Screener,3.0,0.8,0,100.0,3.0,1.1,100.0,0,N/A,1,100.0,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.01,18.7,3,47.4,0.96,19.7,42.9,10,50.0,4,25.0,1.17,16.2,60.0,2,100.0,3,33.3
TRANSITION,1.67,3.3,0,50.0,1.51,3.7,50.0,2,50.0,0,N/A,2.27,2.4,N/A,0,N/A,0,N/A
Attacking Closeouts,1.48,9.6,1,66.7,1.41,10.2,66.7,2,100.0,4,50.0,1.67,8.1,66.7,1,100.0,2,50.0
Catch & Shoot,1.59,13.4,0,52.9,1.59,18.9,52.9,0,N/A,17,52.9,N/A,0.0,N/A,0,N/A,0,N/A


In [34]:
newOppStats = PPP(new_player_data['opp'])
newOppStats.to_csv(f'Opp/PPP Stats/{prevGames[-1] + Opp}.csv')
newOppStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.95,14.1,3,53.3,1.0,10.0,57.1,6,50.0,1,100.0,0.91,22.5,50.0,4,75.0,4,25.0
PNR Screener,1.33,2.2,1,100.0,1.0,2.2,100.0,1,100.0,0,N/A,2.0,2.3,100.0,1,100.0,0,N/A
DHO Ball Handler,0.86,7.0,2,33.3,0.0,5.6,0.0,2,0.0,1,0.0,1.85,9.8,66.7,1,100.0,2,50.0
DHO Screener,1.09,5.5,0,33.3,1.85,4.8,66.7,1,100.0,2,50.0,0.0,6.8,0.0,2,0.0,1,0.0
DBL Ball Handler,0.0,0.7,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,2.3,0.0,0,N/A,1,0.0
DBL Screener,0.0,0.7,0,0.0,0.0,1.1,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.19,21.9,3,54.5,1.08,23.6,52.9,14,64.3,3,0.0,1.46,18.6,60.0,2,100.0,3,33.3
TRANSITION,1.78,14.2,0,75.0,1.52,10.2,71.4,7,71.4,0,N/A,2.02,22.5,77.8,4,75.0,5,80.0
Attacking Closeouts,1.42,7.4,0,55.6,0.87,7.6,33.3,4,50.0,2,0.0,2.67,6.8,100.0,1,100.0,2,100.0
Catch & Shoot,1.73,12.9,0,56.2,1.73,19.2,56.2,0,N/A,16,56.2,N/A,0.0,N/A,0,N/A,0,N/A


In [35]:
TeamSeriesStats = PPP(ovr_series_player_data['team'])
TeamSeriesStats.to_csv(f'Team/PPP Stats/{prevGames[-1] + Opp}.csv')
TeamSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.23,22.1,5,50.0,0.78,20.4,32.7,35,28.6,14,42.9,2.04,26.2,81.5,13,76.9,14,85.7
PNR Screener,1.65,6.4,4,82.4,1.75,7.4,80.0,11,72.7,4,100.0,1.23,4.1,100.0,2,100.0,0,N/A
DHO Ball Handler,0.52,6.7,0,20.0,0.47,4.6,9.1,4,25.0,7,0.0,0.57,11.7,28.6,10,40.0,4,0.0
DHO Screener,0.74,5.5,2,33.3,0.8,3.6,40.0,10,40.0,0,N/A,0.68,9.8,25.0,3,66.7,5,0.0
DBL Ball Handler,1.6,1.3,0,60.0,N/A,0.0,N/A,0,N/A,0,N/A,1.6,4.2,60.0,1,100.0,4,50.0
DBL Screener,2.67,0.8,0,100.0,2.67,1.1,100.0,1,100.0,2,100.0,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.07,17.2,7,50.0,1.03,17.1,47.5,31,51.6,9,33.3,1.14,17.5,55.6,6,100.0,12,33.3
TRANSITION,1.44,4.9,1,64.3,1.42,4.5,66.7,9,66.7,0,N/A,1.48,5.6,60.0,4,75.0,1,0.0
Attacking Closeouts,1.17,9.9,3,53.1,1.09,9.9,50.0,9,66.7,11,36.4,1.33,10.0,58.3,5,100.0,7,28.6
Catch & Shoot,1.38,14.6,0,46.6,1.38,20.9,46.6,1,100.0,57,45.6,N/A,0.0,N/A,0,N/A,0,N/A


In [36]:
OppSeriesStats = PPP(ovr_series_player_data['opp'])
OppSeriesStats.to_csv(f'Opp/PPP Stats/{prevGames[-1] + Opp}.csv')
OppSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.05,15.5,5,54.3,1.0,12.8,55.0,16,62.5,4,25.0,1.12,20.8,53.3,6,83.3,9,33.3
PNR Screener,1.52,3.7,2,85.7,1.6,3.8,80.0,3,100.0,2,50.0,1.33,3.3,100.0,2,100.0,0,N/A
DHO Ball Handler,0.96,6.5,3,41.7,0.48,5.8,25.0,5,40.0,3,0.0,1.67,8.0,75.0,2,100.0,2,50.0
DHO Screener,0.96,5.8,0,30.8,1.94,3.4,75.0,2,100.0,2,50.0,0.32,10.5,11.1,5,20.0,4,0.0
DBL Ball Handler,0.0,0.4,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,1.1,0.0,0,N/A,1,0.0
DBL Screener,1.0,0.7,0,50.0,0.0,0.6,0.0,0,N/A,1,0.0,2.0,1.1,100.0,1,100.0,0,N/A
ISOLATION,1.08,23.0,7,50.0,0.93,22.8,47.1,29,55.2,5,0.0,1.38,23.3,57.1,4,100.0,10,40.0
TRANSITION,1.46,12.7,1,59.3,1.25,10.2,53.8,12,58.3,1,0.0,1.71,17.5,64.3,6,66.7,8,62.5
Attacking Closeouts,1.47,5.8,0,61.5,1.23,5.4,50.0,5,60.0,3,33.3,1.87,6.5,80.0,2,100.0,3,66.7
Catch & Shoot,1.49,12.0,0,48.4,1.49,18.0,48.4,0,N/A,31,48.4,N/A,0.0,N/A,0,N/A,0,N/A


In [37]:
teamStats = PPP(ovr_player_data['team'])
teamStats.to_csv('!PPP Stats/teamStats.csv')
teamStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.09,20.7,28,47.0,0.92,20.1,39.0,144,35.4,66,47.0,1.46,22.2,65.6,47,68.1,43,62.8
PNR Screener,1.18,5.0,21,67.9,1.23,5.7,66.7,40,67.5,8,62.5,1.01,3.4,75.0,3,100.0,5,60.0
DHO Ball Handler,0.61,6.3,4,23.7,0.76,5.2,27.6,19,36.8,39,23.1,0.39,8.9,17.9,26,23.1,13,7.7
DHO Screener,0.81,5.1,4,33.3,0.43,2.3,20.0,24,20.8,1,0.0,0.99,11.9,39.6,20,55.0,33,30.3
DBL Ball Handler,1.01,1.1,0,38.9,0.75,0.3,25.0,2,0.0,2,50.0,1.08,3.0,42.9,4,100.0,10,20.0
DBL Screener,2.33,0.4,0,100.0,2.33,0.5,100.0,4,100.0,2,100.0,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.01,22.7,33,47.1,0.89,23.3,41.4,200,44.5,37,24.3,1.33,21.3,62.2,37,83.8,53,47.2
TRANSITION,1.47,8.3,12,72.8,1.43,6.8,73.4,54,77.8,10,50.0,1.52,11.9,72.0,32,84.4,18,50.0
Attacking Closeouts,1.05,7.8,10,50.0,0.98,7.7,46.1,50,56.0,26,26.9,1.23,8.1,57.9,18,94.4,20,25.0
Catch & Shoot,1.45,12.3,2,50.0,1.44,17.1,49.5,7,100.0,195,47.7,2.43,0.6,100.0,0,N/A,2,100.0


In [38]:
oppStats = PPP(ovr_player_data['opp'])
oppStats.to_csv('!PPP Stats/oppStats.csv')
oppStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.0,14.6,26,47.7,0.84,11.8,41.4,70,47.1,29,27.6,1.19,20.7,55.8,33,78.8,44,38.6
PNR Screener,1.37,4.9,10,66.7,1.48,4.9,70.0,25,84.0,15,46.7,1.1,4.8,57.1,4,75.0,10,50.0
DHO Ball Handler,0.93,3.1,4,37.8,0.81,2.6,34.8,10,50.0,13,23.1,1.1,4.2,42.9,8,50.0,6,33.3
DHO Screener,0.84,2.5,1,31.2,1.22,1.3,40.0,7,42.9,3,33.3,0.63,5.2,27.3,12,41.7,10,10.0
DBL Ball Handler,1.1,0.7,0,50.0,0.67,0.3,33.3,3,33.3,0,N/A,1.29,1.5,57.1,3,100.0,4,25.0
DBL Screener,1.86,0.5,0,85.7,1.8,0.5,80.0,3,100.0,2,50.0,2.0,0.4,100.0,2,100.0,0,N/A
ISOLATION,1.01,21.6,32,47.2,0.95,21.1,46.0,136,53.7,38,18.4,1.13,22.5,50.0,30,76.7,50,34.0
TRANSITION,1.18,14.0,20,53.8,1.13,11.7,53.4,70,61.4,18,22.2,1.24,19.1,54.4,32,78.1,36,33.3
Attacking Closeouts,1.03,10.8,12,44.9,0.94,10.6,42.5,58,50.0,22,22.7,1.21,11.4,48.9,13,84.6,34,35.3
Catch & Shoot,1.03,15.5,3,35.0,1.04,22.1,35.0,2,100.0,215,34.4,0.5,0.9,33.3,1,100.0,2,0.0
